# EdNet Feature extraction

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import dask.dataframe as dd

In [2]:
ednet_path = Path("../EdNet")

In [3]:
ITEM_COL = "item_id"
USER_COL = "user_id"
TIME_COL = "timestamp"
SESSION_COL = "session_id"
CONSECUTIVE_ID = "item_consecutive_id"

## Behaviour pattern extractions
1. Extract all the features in to per session features
2. Merge all features
3. Merge all features to records

In [4]:
kt4_video = pd.read_parquet(ednet_path / "kt4_video_context.parquet")#.drop(columns=["user_answer", "source"])

In [5]:
kt4_video = kt4_video.sort_values(TIME_COL)

In [6]:
kt4_video.isna().any()

tags                   False
item_id                False
views                  False
length                 False
user_id                False
timestamp              False
action_type            False
cursor_time             True
item_consecutive_id    False
back_gap               False
forward_gap            False
session_id             False
dtype: bool

In [7]:
kt4_video

,tags,item_id,views,length,user_id,timestamp,action_type,cursor_time,item_consecutive_id,back_gap,forward_gap,session_id
2177622,87,l524,2256,256000.0,247616,1535380130027,enter,NaN,1,0 days 00:00:00,0 days 00:00:05.038000,0
2177623,87,l524,2256,256000.0,247616,1535380135065,play_video,0.0,1,0 days 00:00:05.038000,0 days 00:00:07.584000,0
2177624,87,l524,2256,256000.0,247616,1535380142649,pause_video,7156.0,1,0 days 00:00:07.584000,0 days 00:00:19.732000,0
2177625,87,l524,2256,256000.0,247616,1535380162381,play_video,7382.0,1,0 days 00:00:19.732000,0 days 00:03:17.418000,0
2177626,87,l524,2256,256000.0,247616,1535380359799,pause_video,204570.0,1,0 days 00:03:17.418000,0 days 00:00:09.290000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4865850,0,l351,2690,184934.0,827192,1574876918267,pause_video,116293.0,7,0 days 00:00:03.056000,0 days 00:00:01.014000,0
4865851,0,l351,2690,184934.0,827192,1574876919281,play_video,106087.0,7,0 days 00:00:01.014000,0 days 00:00:52.577000,0
4865852,0,l351,2690,184934.0,827192,1574876971858,pause_video,158879.0,7,0 days 00:00:52.577000,0 days 00:01:03.953000,0
4865853,0,l351,2690,184934.0,827192,1574877035811,play_video,158946.0,7,0 days 00:01:03.953000,0 days 00:00:16.204000,0


In [8]:
# Session case fixed
kt4_video[(kt4_video[USER_COL] == 356544)&(kt4_video[CONSECUTIVE_ID]==37)][:49]; 

In [97]:
test_group = kt4_video[48:70].groupby([USER_COL, CONSECUTIVE_ID, SESSION_COL])
test_group.apply(lambda group: group)

,tags,item_id,views,length,user_id,timestamp,action_type,cursor_time,item_consecutive_id,back_gap,forward_gap,session_id
3741180,0,l72,1543,293020.0,247535,1535437598252,enter,NaN,3,0 days 00:00:00,0 days 00:00:04.296000,0
3741181,0,l72,1543,293020.0,247535,1535437602548,play_video,0.0,3,0 days 00:00:04.296000,0 days 00:00:00.411000,0
3741182,0,l72,1543,293020.0,247535,1535437602959,pause_video,0.0,3,0 days 00:00:00.411000,0 days 00:00:00.422000,0
3741183,0,l72,1543,293020.0,247535,1535437603381,play_video,160.0,3,0 days 00:00:00.422000,0 days 00:00:01.958000,0
3741184,0,l72,1543,293020.0,247535,1535437605339,pause_video,849.0,3,0 days 00:00:01.958000,0 days 00:00:00.430000,0
3741185,0,l72,1543,293020.0,247535,1535437605769,play_video,1735.0,3,0 days 00:00:00.430000,0 days 00:00:03.641000,0
3741186,0,l72,1543,293020.0,247535,1535437609410,pause_video,2878.0,3,0 days 00:00:03.641000,0 days 00:00:00.054000,0
3741187,0,l72,1543,293020.0,247535,1535437609464,play_video,26700.0,3,0 days 00:00:00.054000,0 days 00:00:04.619000,0
3741188,0,l72,1543,293020.0,247535,1535437614083,pause_video,30647.0,3,0 days 00:00:04.619000,0 days 00:00:00.069000,0
3741189,0,l72,1543,293020.0,247535,1535437614152,play_video,37391.0,3,0 days 00:00:00.069000,0 days 00:00:10.184000,0


In [15]:
test_group_2 = kt4_video[(kt4_video[USER_COL]==287059)&(kt4_video[CONSECUTIVE_ID]==56)&(kt4_video[SESSION_COL]==0)]
test_group_2 = test_group_2.groupby([USER_COL, CONSECUTIVE_ID, SESSION_COL])
test_group_2.apply(lambda group: group)

,tags,item_id,views,length,user_id,timestamp,action_type,cursor_time,item_consecutive_id,back_gap,forward_gap,session_id
833858,191,l870,49,166000.0,287059,1541312707824,enter,NaN,56,0 days 00:00:00,0 days 00:00:07.765000,0
833859,191,l870,49,166000.0,287059,1541312715589,play_video,33.0,56,0 days 00:00:07.765000,0 days 00:00:04.253000,0
833860,191,l870,49,166000.0,287059,1541312719842,pause_video,4286.0,56,0 days 00:00:04.253000,0 days 00:00:11.409000,0
833861,191,l870,49,166000.0,287059,1541312731251,play_video,4341.0,56,0 days 00:00:11.409000,0 days 00:00:27.660000,0
833862,191,l870,49,166000.0,287059,1541312758911,pause_video,26734.0,56,0 days 00:00:27.660000,0 days 00:00:00.371000,0
833863,191,l870,49,166000.0,287059,1541312759282,play_video,26934.0,56,0 days 00:00:00.371000,0 days 00:00:06.489000,0
833864,191,l870,49,166000.0,287059,1541312765771,pause_video,29873.0,56,0 days 00:00:06.489000,0 days 00:00:01.104000,0
833865,191,l870,49,166000.0,287059,1541312766875,play_video,30067.0,56,0 days 00:00:01.104000,0 days 00:00:04.370000,0
833866,191,l870,49,166000.0,287059,1541312771245,pause_video,30076.0,56,0 days 00:00:04.370000,0 days 00:00:01.113000,0
833867,191,l870,49,166000.0,287059,1541312772358,play_video,30272.0,56,0 days 00:00:01.113000,0 days 00:00:32.670000,0


#### Detecting features
**Fast Forward**
1. If Start < end and it is not play -> pause
1. If play -> Pause and the gap is much smaller than difference in cursor time
2. If play -> play and the gap is smaller than the difference in cursor time. If the difference is larger than e.g. 1 sec
3. If pause -> pause and the newest cursor time is higher than the last

**Backwards**
1. If play -> Pause and the gap is much smaller than difference in cursor time
2. If play -> play and the gap is smaller than the difference in cursor time. If the difference is larger than e.g. 1 sec
3. If pause -> pause and the newest cursor time is higher than the last

In [ ]:
def get_num_ff(group, cursor_col="cursor_time", action_col="action_type"):
    """NF
    Ignoring global timestamps and speeds. Removing border events due
    to inconsistencies in tracking, i.e. sometimes placed before last pause
    also in Brinton PLA"""
    group_clean = group[~group["action_type"].isin(["enter", "quit"])]
    group_shifted = group_clean.shift(-1)
    is_forward = group_clean[cursor_col] < group_shifted[cursor_col]
    is_linear = (group[action_col] == "play_video")&(group_shifted[action_col] == "pause_video")
    return (is_forward&(~is_linear)).sum()

In [217]:
# teST SUITE for kt4_video_sorted[48:70]
assert ((test_num_ff := test_group.apply(get_num_ff)) == 9).all(), f"Number of forward seeks should be 9, was {test_num_ff.values[0]}"
assert ((test_num_pauses := test_group.apply(get_num_pauses)) == 10).all(), f"Number of pauses should be 10, was {test_num_pauses.values[0]}"
assert ((test_pause_dur := test_group.apply(get_median_pause_dur)) < pd.Timedelta("00:00:01")).all(), f"Median pause duration should be less than 1s, was {test_pause_dur.values[0].astype('timedelta64[ms]')}"
assert ((test_num_backwards := test_group.apply(get_num_backward_seeks)) == 1).all(), f"Number of backwards seeks should be 1, was {test_num_backwards.values[0]}"
exp_frac_comp = sum([689, 1143, 3947, 8908, 30, 1914, 199959, 7364]) / 293020
assert ((test_frac_comp := test_group.apply(get_interval_based_feature, interval_feat_extract_fn=get_frac_comp)) == exp_frac_comp).all(), f"Number of backwards seeks should be {exp_frac_comp}, was {test_frac_comp.values[0]}"

            start       end
3741181       0.0       0.0
3741183     160.0     849.0
3741185    1735.0    2878.0
3741187   26700.0   30647.0
3741189   37391.0   46299.0
3741191   58706.0    4873.0
3741193   48023.0   48053.0
3741195   69383.0   71297.0
3741197   85359.0  285318.0
3741199  285440.0  292804.0
FRAC COMP IS CALLED             start       end
3741181       0.0       0.0
3741183     160.0     849.0
3741185    1735.0    2878.0
3741187   26700.0   30647.0
3741189   37391.0   46299.0
3741191   58706.0    4873.0
3741193   48023.0   48053.0
3741195   69383.0   71297.0
3741197   85359.0  285318.0
3741199  285440.0  292804.0 293020.0
          start       end
group                    
0         160.0     849.0
1        1735.0    2878.0
2       26700.0   30647.0
3       37391.0   46299.0
4       48023.0   48053.0
5       69383.0   71297.0
6       85359.0  285318.0
7      285440.0  292804.0


In [90]:
# teST SUITE for test group 2
assert ((test_num_pauses_2 := test_group_2.apply(get_num_pauses)) == 14).all(), f"Number of pauses should be 14, was {test_num_pauses_2.values[0]}"
assert ((test_num_ff_2 := test_group_2.apply(get_num_ff)) == 15).all(), f"Number of forward seeks should be 15, was {test_num_ff_2.values[0]}"
assert ((test_num_backwards_2 := test_group_2.apply(get_num_backward_seeks)) == 3).all(), f"Number of backwards seeks should be 3, was {test_num_backwards_2.values[0]}"
assert ((test_pause_dur_2 := test_group_2.apply(get_median_pause_dur)) > pd.Timedelta("00:00:01")).all(), f"Median pause duration should be more than 1s, was {test_pause_dur_2.values[0].astype('timedelta64[ms]')}"


In [61]:
def get_num_backward_seeks(group, cursor_col="cursor_time"):
    """Only basing on cursor time, since we don't have the playtime speed"""
    group_clean = group[~group["action_type"].isin(["enter", "quit"])]
    intervals = pd.concat([group_clean[cursor_col].rename("start"), group_clean[cursor_col].shift(-1).rename("end")], axis=1)[:-1]
    print(intervals)
    print((group_clean[cursor_col] > group_clean[cursor_col].shift(-1)).sum())
    return group
    #for idx, row in group.iterrows():
        
    

In [91]:
kt4_video[(kt4_video[USER_COL]==459219)][120:150]

,tags,item_id,views,length,user_id,timestamp,action_type,cursor_time,item_consecutive_id,back_gap,forward_gap,session_id
385239,4,l661,247,243000.0,459219,1560625594967,enter,NaN,19,0 days 00:00:00,0 days 00:00:02.757000,0
385240,4,l661,247,243000.0,459219,1560625597724,quit,NaN,19,0 days 00:00:02.757000,0 days 00:00:00,0
385241,4,l661,247,243000.0,459219,1560625597724,play_video,192.0,19,0 days 00:00:00,0 days 00:04:11.267000,0
385242,4,l661,247,243000.0,459219,1560625848991,play_video,3381.0,19,0 days 00:04:11.267000,0 days 00:00:00.003000,0
385243,4,l661,247,243000.0,459219,1560625848994,pause_video,3326.0,19,0 days 00:00:00.003000,0 days 00:00:00.544000,0
385244,4,l661,247,243000.0,459219,1560625849538,play_video,3523.0,19,0 days 00:00:00.544000,0 days 00:17:34.374000,0
385245,4,l661,247,243000.0,459219,1560626903912,play_video,14882.0,19,0 days 00:17:34.374000,0 days 00:00:00,0
389832,10,l664,144,272000.0,459219,1560627134971,play_video,27.0,20,0 days 00:00:00,0 days 00:02:04.229000,0
389833,10,l664,144,272000.0,459219,1560627259200,pause_video,124178.0,20,0 days 00:02:04.229000,0 days 00:01:12.613000,0
389834,10,l664,144,272000.0,459219,1560627331813,play_video,124215.0,20,0 days 00:01:12.613000,0 days 00:01:20.346000,0


In [45]:
# Sessionify did not handle consecutive play_video
# Should perhaps only ignore play->pause?
kt4_video[(kt4_video[USER_COL]==459219)&(kt4_video[CONSECUTIVE_ID]==19)&(kt4_video[SESSION_COL]==0)]

,tags,item_id,views,length,user_id,timestamp,action_type,cursor_time,item_consecutive_id,back_gap,forward_gap,session_id
385239,4,l661,247,243000.0,459219,1560625594967,enter,NaN,19,0 days 00:00:00,0 days 00:00:02.757000,0
385240,4,l661,247,243000.0,459219,1560625597724,quit,NaN,19,0 days 00:00:02.757000,0 days 00:00:00,0
385241,4,l661,247,243000.0,459219,1560625597724,play_video,192.0,19,0 days 00:00:00,0 days 00:04:11.267000,0
385242,4,l661,247,243000.0,459219,1560625848991,play_video,3381.0,19,0 days 00:04:11.267000,0 days 00:00:00.003000,0
385243,4,l661,247,243000.0,459219,1560625848994,pause_video,3326.0,19,0 days 00:00:00.003000,0 days 00:00:00.544000,0
385244,4,l661,247,243000.0,459219,1560625849538,play_video,3523.0,19,0 days 00:00:00.544000,0 days 00:17:34.374000,0
385245,4,l661,247,243000.0,459219,1560626903912,play_video,14882.0,19,0 days 00:17:34.374000,0 days 00:00:00,0


In [36]:
test_num = -1
test_thresh = pd.Timedelta("00:00:01")
subdf = kt4_video[:test_num]
test_type = subdf["action_type"]
both_play = (test_type == "play_video")&(test_type.shift(-1) == "play_video")
same_session = (subdf[USER_COL] == subdf[USER_COL].shift(-1))&(subdf[CONSECUTIVE_ID] == subdf[CONSECUTIVE_ID].shift(-1))&(subdf[SESSION_COL] == subdf[SESSION_COL].shift(-1))
consec_plays = subdf[both_play&same_session&(subdf["forward_gap"] > test_thresh)].sort_values("forward_gap", ascending=False)

In [51]:
# Number of sessions with less than 4 interactions (idea: enter, play, pause, quit)
session_interaction_count = kt4_video.groupby([USER_COL,CONSECUTIVE_ID,SESSION_COL]).size()
session_interaction_count.shape[0] - session_interaction_count[session_interaction_count > 3].shape[0]

32818

In [46]:
consec_plays

,tags,item_id,views,length,user_id,timestamp,action_type,cursor_time,item_consecutive_id,back_gap,forward_gap,session_id
385241,4,l661,247,243000.0,459219,1560625597724,play_video,192.0,19,0 days 00:00:00,0 days 00:04:11.267000,0
2152750,116,l523,2826,236000.0,328232,1543294791555,play_video,43976.0,4,0 days 00:00:00,0 days 00:03:04.174000,1
2840615,88,l754,1651,206000.0,5946,1569527426805,play_video,15149.0,53,0 days 00:00:00.533000,0 days 00:03:02.425000,1
570552,50,l449,2575,224000.0,395504,1559479850690,play_video,81902.0,52,0 days 00:00:20.557000,0 days 00:02:00.170000,0
3086627,134,l800,696,145000.0,3418,1562438769314,play_video,3236.0,41,0 days 00:00:04.841000,0 days 00:01:51.963000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3659738,0,l57,205,392712.0,3361,1573441804381,play_video,202640.0,116,0 days 00:00:00.042000,0 days 00:00:01.009000,0
2044127,72,l517,2387,207000.0,367815,1573138588273,play_video,140036.0,5,0 days 00:00:00.010000,0 days 00:00:01.007000,0
4230741,0,l229,1404,287481.0,17696,1570113497982,play_video,4029.0,8,0 days 00:00:00.056000,0 days 00:00:01.006000,0
4768810,0,l334,535,132090.0,818531,1574528365098,play_video,66241.0,1,0 days 00:00:00.098000,0 days 00:00:01.001000,0


In [86]:
## load data
import io

raw ="""START,FINISH
0.000000    ,10.000000
10.000000   ,4500.182997
5000.00    ,7000.000000
6000   ,8500.687227
9850.123,9990.000000
"""

buf_bytes = io.StringIO(raw)
df=pd.read_csv(buf_bytes)

## solution

df.sort_values("START", inplace=True)

## This line compares if START of next row is greater than FINISH of current
## row ("shift" shifts down FINISH by one row). The value of expression before
## cumsum will be True if interval breaks (i.e. cannot be merged), so  
## cumsum will increment group value when interval breaks (cum sum treats True=1, False=0)
df["group"]=(df["START"]>df["FINISH"].shift()).cumsum()

## this returns min value of "START" column from a group and max value fro m "FINISH"
result=df.groupby("group").agg({"START":"min", "FINISH": "max"})
display(result)

,START,FINISH
group,,
0,0.000,4500.182997
1,5000.000,8500.687227
2,9850.123,9990.000000


In [87]:
df["FINISH"].shift()

0            NaN
1      10.000000
2    4500.182997
3    7000.000000
4    8500.687227
Name: FINISH, dtype: float64

In [81]:
df

,START,FINISH,group
0,0.000,10.000000,0
1,10.000,4500.182997,0
2,5000.000,7000.000000,1
3,6000.000,8500.687227,1
4,9850.123,9990.000000,2


In [220]:
def get_interval_based_feature(group, cursor_col="cursor_time", action_col="action_type",
                               interval_feat_extract_fn=lambda : np.na, **kwargs):
    """Method for making watched intervals from the records.
    Should return the extracted feature, often a singular row pd.Series
    NB: Ignoring consecutive play and pause events, which do happen sometimes"""
    # If the current row is play and next is pause OR current row is paus and previous was play
    is_linear = (((group[action_col] == "play_video")&(group[action_col].shift(-1) == "pause_video"))|
                      ((group[action_col] == "pause_video")&(group[action_col].shift() == "play_video")))
    group_clean = group[is_linear]
    
    # Create intervals of play-> pause sequences, ignoring pause-> play sequences
    intervals = pd.concat([group_clean[cursor_col].rename("start"), group_clean[cursor_col].shift(-1).rename("end")], axis=1)[::2]
    return interval_feat_extract_fn(intervals, group["length"].values[0], **kwargs)


#### Features from Behaviour Pattern and difficulty

In [12]:
def get_num_pauses(group, action_col="action_type", pause_label="pause_video"):  
    """NP, currently only for EdNet"""
    # Assuming few consecutive pause actions, though some, i.e. when scrubbing -> Not actually a pause
    return (group[action_col] == pause_label).sum()

# Should add FORWARD GAP feature before
def get_median_pause_dur(group, action_col="action_type", pause_label="pause_video", pause_col="forward_gap"):
    """MP"""
    # Calculating forward_pause - Removed when complete
    group[pause_col] = group[TIME_COL].shift(-1) - group[TIME_COL]
    return pd.to_timedelta(group.loc[group[action_col] == pause_label, pause_col].median(), unit="ms").round("100ms")

def get_num_ff(group, cursor_col="cursor_time", action_col="action_type"):
    """NF
    Ignoring global timestamps and speeds. Removing border events due
    to inconsistencies in tracking, i.e. sometimes placed before last pause.
    Do count consecutive play events for instance, though might not be entirely correct
    also in Brinton PLA"""
    group_clean = group[~group["action_type"].isin(["enter", "quit"])]
    group_shifted = group_clean.shift(-1)
    is_forward = group_clean[cursor_col] < group_shifted[cursor_col]
    is_linear = (group[action_col] == "play_video")&(group_shifted[action_col] == "pause_video")
    return (is_forward&(~is_linear)).sum()


def get_num_backward_seeks(group, cursor_col="cursor_time"):
    """NB - Important factor
    Only basing on cursor time, since we don't have the playtime speed
    Also in Brinton PLA"""
    #Might have to ignore enter or quits because they get in the way
    group_clean = group[~group["action_type"].isin(["enter", "quit"])]
    return (group_clean[cursor_col] > group_clean[cursor_col].shift(-1)).sum()

def get_frac_skipped(group):
    """SR
    Actually skipped and not watched.
    So finding the intervals pause -> play, and the usual intervals play-> pause
    Subtracting the usual plays from the skipped intervals, as they are seen
    """
    pass

def get_replayed_video_length(group):
    """RL - Important factor"""
    pass

def get_average_speed(group):
    """AS, not applicable to EdNet"""
    return (group["duration"] * group["speed"]).sum() / group["duration"].sum()

def get_effective_speed(group):
    """SC - Average speed - minus start speed. Not relevant for EdNet
    I believe start speed is not carried over for MOOC"""
    pass

#### Brinton PLA

In [19]:
def get_frac_comp(intervals, duration, start_col="start", end_col="end"):
    """Proportion of video completed, 0-1, not counting repeated segments."""
    forward_intervals = intervals[intervals[start_col] < intervals[end_col]]
    # Create ids for non-consecutive intervals
    forward_intervals["group"]=(forward_intervals["start"] > forward_intervals["end"].shift()).cumsum()
    ## this returns min value of "START" column from a group and max value fro m "FINISH"
    non_consecutive_intervals = forward_intervals.groupby("group").agg({start_col:"min", end_col: "max"})
    return (non_consecutive_intervals[end_col] - non_consecutive_intervals[start_col]).sum() / duration

def get_frac_spent(group):
    """Total time spent on a video, including pauses -> > 1, divided by video length.
    Abnormally high -> Could indicate off-task behaviour"""
    pass

def get_frac_played(group):
    """Like get_frac_comp, but counting repeated segments -> >= 1"""
    pass

def get_frac_paused(group):
    """Amount of time pausing relative to video length, >= 1
    Abnormally high -> Could indicate off-task behaviour"""
    # Probably not
    pass

def get_std_speed(group):
    """Standard deviation of video speed"""
    pass

#### AI and Educational Psychology
Not relevant as item features are not currently extracted,
as well as this would have to be time based on a global scale which seems difficult to calculate efficiently.

The other features are the same as with Brinton
**Notes**
- Pauses are not used as they can be caused by external circumstances
- Use NumRW for video difficulty calculation instead of NumFF

In [20]:
# Video features
def get_frac_study(video):
    """Number of students who've interacted with the video / total number of students""" 
    pass

def get_avg_frac_comp(video):
    """The average completion ratio of the video based on students who've watched it"""
    pass

def get_frac_num_backward_seeks(video):
    """The average number of backward seeks, 
    where the number of students (denominator) 
    is multiplied with the maximum backward seek value"""
    pass

#### Custom

In [ ]:
def get_segment_repetition(group):
    pass

In [ ]:
# DEPRECATED
def get_frac_comp(group, duration="length"):
    """Proportion of video completed, 0-1, not counting repeated segments.
    """
    # If the current row is play and next is pause OR current row is paus and previous was play
    is_linear = (((group[action_col] == "play_video")&(group[action_col].shift(-1) == "pause_video"))|
                      ((group[action_col] == "pause_video")&(group[action_col].shift() == "play_video")))
    group_clean = group[is_linear]
    
    # Create intervals of play-> pause sequences
    intervals = pd.concat([group_clean[cursor_col].rename("start"), group_clean[cursor_col].shift(-1).rename("end")], axis=1)[::2]
    
    #---- Feature specific cleaning
    # Remove same-timepoint intervals + invalid intervals  - Could now create IntervalArray
    intervals=intervals[intervals["start"] < intervals["end"]]
    
    # Create ids for non-consecutive intervals
    intervals["group"]=(intervals["start"]>intervals["end"].shift()).cumsum()

    ## this returns min value of "START" column from a group and max value fro m "FINISH"
    non_consecutive_intervals = intervals.groupby("group").agg({"start":"min", "end": "max"})
    return (non_consecutive_intervals["end"] - non_consecutive_intervals["start"]).sum() / group[duration].values[0]

In [185]:
# DEPRECATED - More use cases for the one above
def get_frac_comp_2(group,  cursor_col="cursor_time", action_col="action_type", duration="length"):
    """Proportion of video completed, 0-1, not counting repeated segments
    """
    # If the current row is play and next is pause OR current row is paus and previous was play
    is_linear = (((group[action_col] == "play_video")&(group[action_col].shift(-1) == "pause_video"))|
                      ((group[action_col] == "pause_video")&(group[action_col].shift() == "play_video")))
    group_clean = group[is_linear]
    # Select the pause durations
    group_diff = group_clean.assign(diff=group_clean[cursor_col].diff())[1::2]
    # Only play->pause which are not backward seeks
    return group_diff[group_diff["diff"] > 0]["diff"].sum() / group[duration].values[0]